# Georeferenciación Galeria Inmobiliaria

## Cargue de librerías

En esta sección se realiza el cargue de las librerías necesarias para el procesamiento de los datos

In [0]:
# !pip install pandas
# !pip install geopandas
# !pip install matplotlib
# !pip install requests
# !pip install swifter
# !pip install pyxlsb
# !pip install openpyxl

!pip install --upgrade pip
!pip install pyxlsb
!pip install pandas
!pip install geopandas

!pip install azure-storage-blob


In [0]:
# import pandas as pd
# import requests
# import json
# import time
# import swifter
# import geopandas as gpd
# from shapely.geometry import Point
# import shapely.wkt
# import matplotlib
# import unicodedata

import pandas as pd
import geopandas as geop
import matplotlib
from shapely.geometry import Point
from shapely.wkt import loads
import unicodedata
import requests
import json
import re

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, struct, from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<command-1496039999229371>", line 27, in <module>
    from azure.storage import BlobServiceClient
  File "/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py", line 171, in import_patch
    original_result = python_builtin_import(name, globals, locals, fromlist, level)
ModuleNotFoundError: No module named 'azure.storage'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 1997, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1112, in structured_traceback
    return FormattedTB.structured_traceback(
 

## Configuración y parametrización

En esta sección se definen los parámetros básicos de visualización de los dataframes, se establecen las direcciones de los servicios web a consumir y se definen las variables para establecer las hojas de interés dentro de cada archivo y los campos a georeferenciar en cada caso.

In [0]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)
num_partitions = 4
spark = SparkSession.builder.getOrCreate()

In [0]:
#URL geocodificador
url = "https://catalogopmb.catastrobogota.gov.co/PMBWeb/web/api"

In [0]:
path_vivienda_usada = 'https://emb.blob.core.windows.net/galeria/Bogota_Programa_Vivienda_Usada_05-23.xlsb'
sheets_vivienda_usada = ['Sop. Venta', 'Sop. Arriendos']
vivienda_usada_dir = 'Dirección Nueva' 
path_comercio = 'https://emb.blob.core.windows.net/galeria/Bogota_Comercio_Def_05-23.xlsb'
sheets_comercio = ['Base Proyecto', 'Base Inmuebles', 'Base Historia']
comercio_dir = 'Dirección'
path_def = 'https://emb.blob.core.windows.net/galeria/Bogota_Def_05-23.xlsb'
sheets_def = ['Base Proyectos']
def_dir = 'Dirección'

In [0]:
%fs mkdirs /tmp/galeria

res0: Boolean = true

## Cargue de insumos

En esta sección se realiza el cargue de los diferentes insumos para su posterior procesamiento. Adicionalmente, se parametrizan las funciones de cargue a partir de la estructura y definición de cada una de las fuentes.

In [0]:
df_vivienda_usada_venta = pd.read_excel(path_vivienda_usada, sheet_name=sheets_vivienda_usada[0], skiprows=4, engine='pyxlsb')
df_vivienda_usada_venta.to_csv('/dbfs/tmp/galeria/df_vivienda_usada_venta.csv', index=False)

---------------------------------------------------------------------------
OSError                                   Traceback (most recent call last)
File <command-1496039999229379>:5
      1 #df_vivienda_usada_venta = pd.read_excel(path_vivienda_usada, sheet_name=sheets_vivienda_usada[0], skiprows=4)
      2 
      3 # Convert .xlsb to CSV using pandas
      4 df_vivienda_usada_venta = pd.read_excel(path_vivienda_usada, sheet_name=sheets_vivienda_usada[0], skiprows=4, engine='pyxlsb')
----> 5 df_vivienda_usada_venta.to_csv('dbfs:/tmp/galeria/df_vivienda_usada_venta.csv', index=False)

File /databricks/python/lib/python3.9/site-packages/pandas/core/generic.py:3551, in NDFrame.to_csv(self, path_or_buf, sep, na_rep, float_format, columns, header, index, index_label, mode, encoding, compression, quoting, quotechar, line_terminator, chunksize, date_format, doublequote, escapechar, decimal, errors, storage_options)
   3540 df = self if isinstance(self, ABCDataFrame) else self.to_frame()
 

In [0]:
df_vivienda_usada_venta_sp = spark.read.csv('dbfs:/tmp/galeria/df_vivienda_usada_venta.csv', header=True)

In [0]:
df_vivienda_usada_arriendo = pd.read_excel(path_vivienda_usada, sheet_name=sheets_vivienda_usada[1], skiprows=4, engine='pyxlsb')
df_vivienda_usada_arriendo.to_csv('/dbfs/tmp/galeria/df_vivienda_usada_arriendo.csv', index=False)

In [0]:
df_vivienda_usada_arriendo_sp = spark.read.csv('dbfs:/tmp/galeria/df_vivienda_usada_arriendo.csv', header=True)

In [0]:
df_comercio_base_proyecto = pd.read_excel(path_comercio, sheet_name=sheets_comercio[0], engine='pyxlsb')
df_comercio_base_proyecto.to_csv('/dbfs/tmp/galeria/df_comercio_base_proyecto.csv', index=False)

In [0]:
df_comercio_base_proyecto_sp = spark.read.csv('dbfs:/tmp/galeria/df_comercio_base_proyecto.csv', header=True)

In [0]:
df_comercio_base_inmuebles = pd.read_excel(path_comercio, sheet_name=sheets_comercio[1], engine='pyxlsb')
df_comercio_base_inmuebles = df_comercio_base_inmuebles.drop([0])
df_comercio_base_inmuebles.to_csv('/dbfs/tmp/galeria/df_comercio_base_inmuebles.csv', index=False)

In [0]:
df_comercio_base_inmuebles_sp = spark.read.csv('dbfs:/tmp/galeria/df_comercio_base_inmuebles.csv', header=True)

In [0]:
df_comercio_base_historia = pd.read_excel(path_comercio, sheet_name=sheets_comercio[2], engine='pyxlsb')
df_comercio_base_historia.to_csv('/dbfs/tmp/galeria/df_comercio_base_historia.csv', index=False)

In [0]:
df_comercio_base_historia_sp = spark.read.csv('dbfs:/tmp/galeria/df_comercio_base_historia.csv', header=True)

In [0]:
df_def_base_proyecto = pd.read_excel(path_def, sheet_name=sheets_def[0], engine='pyxlsb')
df_def_base_proyecto.to_csv('/dbfs/tmp/galeria/df_def_base_proyecto.csv', index=False)

In [0]:
df_def_base_proyecto_sp = spark.read.csv('dbfs:/tmp/galeria/df_def_base_proyecto.csv', header=True)

In [0]:
%fs ls /tmp/galeria/

path,name,size,modificationTime
dbfs:/tmp/galeria/df_comercio_base_historia.csv,df_comercio_base_historia.csv,2853151,1688431570000
dbfs:/tmp/galeria/df_comercio_base_inmuebles.csv,df_comercio_base_inmuebles.csv,32486290,1688431563000
dbfs:/tmp/galeria/df_comercio_base_proyecto.csv,df_comercio_base_proyecto.csv,693954,1688431478000
dbfs:/tmp/galeria/df_def_base_proyecto.csv,df_def_base_proyecto.csv,2088789,1688431575000
dbfs:/tmp/galeria/df_vivienda_usada_arriendo.csv,df_vivienda_usada_arriendo.csv,9584513,1688431416000
dbfs:/tmp/galeria/df_vivienda_usada_venta.csv,df_vivienda_usada_venta.csv,11269151,1688431175000


## Funciones

En esta sección se definen las funciones que se van a utilizar para el procesamiento de los datos

### Función georeferenciar

La función georeferenciar se encarga de obtener a partir del uso de un servicio web externo las coordenadas geográficas de una dirección entregada por parámetro.

In [0]:
def remove_words_from_string(input_str, words_to_remove):
    for word in words_to_remove:
        input_str = input_str.replace(word, "")
    return input_str

schema = StructType([
                    StructField("estado", StringType()),
                    StructField("yinput", DoubleType()),
                    StructField("lotcodigo", StringType()),
                    StructField("latitude", StringType()),
                    StructField("diraprox", StringType()),
                    StructField("mancodigo", StringType()),
                    StructField("cpocodigo", StringType()),
                    StructField("xinput", DoubleType()),
                    StructField("codloc", StringType()),
                    StructField("dirtrad", StringType()),
                    StructField("nomupz", StringType()),
                    StructField("localidad", StringType()),
                    StructField("dirinput", StringType()),
                    StructField("codupz", StringType()),
                    StructField("nomseccat", StringType()),
                    StructField("tipo_direccion", StringType()),
                    StructField("codseccat", StringType()),
                    StructField("longitude", StringType()),
                ])
def georeferenciar(input_str:str):
    schema = StructType([
                    StructField("estado", StringType()),
                    StructField("yinput", DoubleType()),
                    StructField("lotcodigo", StringType()),
                    StructField("latitude", StringType()),
                    StructField("diraprox", StringType()),
                    StructField("mancodigo", StringType()),
                    StructField("cpocodigo", StringType()),
                    StructField("xinput", DoubleType()),
                    StructField("codloc", StringType()),
                    StructField("dirtrad", StringType()),
                    StructField("nomupz", StringType()),
                    StructField("localidad", StringType()),
                    StructField("dirinput", StringType()),
                    StructField("codupz", StringType()),
                    StructField("nomseccat", StringType()),
                    StructField("tipo_direccion", StringType()),
                    StructField("codseccat", StringType()),
                    StructField("longitude", StringType()),
                ])
    """
    Realiza la georreferenciación de una cadena de entrada utilizando una API de geocodificación.

    Args:
        input_str (str): La cadena a georreferenciar.

    Returns:
        dict: Un diccionario que contiene los datos de georreferenciación si la operación fue exitosa.
              En caso contrario, devuelve un diccionario que indica un estado de falla o un estado de error.

    """
    direc = unicodedata.normalize("NFKD", input_str).encode("ascii","ignore").decode("ascii").lower()

    val = [True for t in direc.replace('.',' ').replace("  "," ").split() if t in ['via','km','chia','cota','tocancipa','tenjo','madrid','tabio,','cajica,','mosquera','zipaquira']]
    
    if sum(val)>=1:
        result = {'estado': 'outside_bogota','yinput':0,'lotcodigo':'','latitude':'','diraprox':'','mancodigo':'','cpocodigo':'','xinput':0,'codloc':'','dirtrad':'','nomupz':'','localidad':'','dirinput':'','codupz':'','nomseccat':'','tipo_direccion':'','codseccat':'','longitude':''}
        return result

    else:
        try:
            
            words_to_remove = ["costado occ.", "esquina"]

            direc = remove_words_from_string(direc, words_to_remove)

            pattern = r'av\.?(?: de la)? esp\.?|\bavenida la esperanza\b|\besperanza\b'
            replacement_str = "AC 24"
            output_str = re.sub(pattern, replacement_str, direc)

            pattern = r'av\.?(?: de las)? ame\.?|\bavenida de las americas\b|\bamericas\b'
            replacement_str = "AC 9"
            output_str = re.sub(pattern, replacement_str, output_str)

            pattern = r'(autop\.?(\s)?norte|autonorte|au\.?(\s)?norte|aut\.?(\s)?norte)'
            replacement_str = "AK 45"
            output_str = re.sub(pattern, replacement_str, output_str)

            pattern = r'av\.?(?: de las)?(?: c\.?)?\s*(?:ciudad\s*)?cali|avenida\s*cali'
            replacement_str = "AK 86"
            output_str = re.sub(pattern, replacement_str, output_str)

            pattern = r'av. nqs|\bnqs\b'
            replacement_str = "AK 30"
            output_str = re.sub(pattern, replacement_str, output_str)

            pattern = r'av. suba|\bsuba\b|tr. suba'
            replacement_str = "AK 69"
            output_str = re.sub(pattern, replacement_str, output_str)

            output_str = output_str.replace('cl. av.', 'AC')
            output_str = output_str.replace('av. cl.', 'AC')
            output_str = output_str.replace('av. cr.', 'AK')
            

            con = False

            output_str = output_str.replace('con','#')
            output_str = output_str.replace('paralela','#')

            parts = output_str.split("#")

            if len(parts) > 1:
                if '-' not in parts[1]:
                    parts[1] = re.sub(r'[a-zA-Z.]', '', parts[1])
                    output_str = "#".join(parts)

            

            pattern = r'(\d+)\D+(\w+)'
            output_str = re.sub(pattern, lambda x: x.group(1) + ' # ' + x.group(2), output_str)

            if '-' not in output_str:
                output_str = output_str + ' - 01'


            parts = output_str.split("#")

            if parts[0].strip() == 'AK 30' and (int(parts[1].split('-')[0].strip())>100):
                parts[0] = 'AK 9'
                output_str = "#".join(parts)

            output_str = output_str.replace("AK 69",'Av Suba')

            print(output_str)

            params = {
                'cmd': 'geocodificar',
                'apikey': '1c025f92-4520-49c1-90a7-a24b3d9d374c',
                'query': output_str
            }

            response = requests.get(url, params=params)
            res = response.json()
            success = res['response']['success']
            if success:
                result = res['response']['data']
                return result
            else:
                result = {'estado': 'failed','yinput':0,'lotcodigo':'','latitude':'','diraprox':'','mancodigo':'','cpocodigo':'','xinput':0,'codloc':'','dirtrad':'','nomupz':'','localidad':'','dirinput':'','codupz':'','nomseccat':'','tipo_direccion':'','codseccat':'','longitude':''}
                return result
        except:
            result = {'estado': 'error_ws','yinput':0,'lotcodigo':'','latitude':'','diraprox':'','mancodigo':'','cpocodigo':'','xinput':0,'codloc':'','dirtrad':'','nomupz':'','localidad':'','dirinput':'','codupz':'','nomseccat':'','tipo_direccion':'','codseccat':'','longitude':''}
            
            return result
        
georeferenciar_udf = udf(georeferenciar, returnType=schema)

In [0]:
def distance(p1: str, p2: str):
    """
    Calcula la distancia entre dos puntos geográficos.

    Args:
        p1 (Point): El primer punto geográfico en formato `Point`.
        p2 (Point): El segundo punto geográfico en formato `Point`.

    Returns:
        Series: Una serie que contiene la distancia entre los dos puntos en la proyección UTM.

    """
    try:
        gdf_1 = geop.GeoDataFrame(geometry=[loads(p1)], crs="EPSG:4326")
        gdf_2 = geop.GeoDataFrame(geometry=[loads(p2)], crs="EPSG:4326")
        utm = gdf_1.estimate_utm_crs()
        return gdf_1.to_crs(utm).distance(gdf_2.to_crs(utm))
    except:
        return 0;

distance_udf = udf(distance, DoubleType())

In [0]:
create_point_udf = udf(lambda x,y: Point(x,y).wkt, returnType=StringType())

In [0]:
'''
Dirección
11	Av. NQS Cl. 101
12	Av. NQS Cl. 101
16	Tr. Suba Cr. 101
18	Cl. Av. 100 # 13-76
24	Av. Cl. 80 # 66A-48
25	Av. Cl. 80 # 66A-48
26	Av. Cl. 80 # 66A-48
32	Tr. Suba Cr. 101
33	Tr. Suba Cr. 101
34	Tr. Suba Cr. 101
62	Dg. 76 # 23H-96F
66	A 900 mts de Aut. Medellín
67	A 900 mts de Aut. Medellín
68	A 900 mts de Aut. Medellín
69	A 900 mts de Aut. Medellín
76	Av. Suba Cl. 100
77	Av. Suba Cl. 100
78	Av. Suba Cl. 100
79	Av. Suba Cl. 100'''

georeferenciar('Tr. Suba Cr. 101')

Av Suba # 101 - 01
Out[458]: {'estado': 'success',
 'yinput': 4.69240921400001,
 'lotcodigo': '005403072099',
 'latitude': '4.69240921400001',
 'diraprox': 'KR 69 101 01',
 'mancodigo': '005403072',
 'cpocodigo': '111121',
 'xinput': -74.073097513,
 'codloc': '11',
 'dirtrad': 'KR 69 101 01',
 'nomupz': 'LA FLORESTA',
 'localidad': 'SUBA',
 'dirinput': 'Av Suba # 101 - 01',
 'codupz': 'UPZ25',
 'nomseccat': 'JULIO FLOREZ',
 'tipo_direccion': 'Dirección por aproximación',
 'codseccat': '005403',
 'longitude': '-74.073097513'}

## Exploración

Para utilizar la función de georeferenciación se va a realizar un muestreo aleatorio de direcciones con el fin de evaluar la calidad de el API geográfico. Esta exploración va a permitir identificar que tipo de ajustes se deben realizar a las direcciones de las bases de datos a georeferenciar.

In [0]:
sample_1 = df_vivienda_usada_venta_sp.sample(withReplacement=False, fraction=0.1, seed=1).limit(150)

In [0]:
sample_1 = sample_1.withColumn('geocode_result', struct(georeferenciar_udf(col(vivienda_usada_dir)).alias('geocoding')))

In [0]:
sample_1 = sample_1.withColumn('estado', col('geocode_result.geocoding.estado'))
sample_1 = sample_1.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
sample_1 = sample_1.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
sample_1 = sample_1.withColumn('lat', col('geocode_result.geocoding.yinput'))
sample_1 = sample_1.withColumn('lng', col('geocode_result.geocoding.xinput'))


In [0]:
# Calculate value counts and normalize
value_counts = sample_1.groupBy('estado').count()
total_count = sample_1.count()
normalized_counts = value_counts.withColumn('normalized_count', col('count') / total_count)

# Display the result
normalized_counts.show()

+--------+-----+--------------------+
|  estado|count|    normalized_count|
+--------+-----+--------------------+
| success|  148|  0.9866666666666667|
|  failed|    1|0.006666666666666667|
|error_ws|    1|0.006666666666666667|
+--------+-----+--------------------+



El 0.6% de la muestra aleatoria del primer dataframe (df_vivienda_usada_venta) no pudo ser georeferenciado por la existencia de detalles en la dirección utilizada como insumo.

In [0]:
sample_2 = df_vivienda_usada_arriendo_sp.sample(withReplacement=False, fraction=0.1, seed=1).limit(150)

In [0]:
sample_2 = sample_2.withColumn('geocode_result', struct(georeferenciar_udf(col(vivienda_usada_dir)).alias('geocoding')))

In [0]:
sample_2 = sample_2.withColumn('estado', col('geocode_result.geocoding.estado'))
sample_2 = sample_2.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
sample_2 = sample_2.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
sample_2 = sample_2.withColumn('lat', col('geocode_result.geocoding.yinput'))
sample_2 = sample_2.withColumn('lng', col('geocode_result.geocoding.xinput'))


In [0]:
# Calculate value counts and normalize
value_counts = sample_2.groupBy('estado').count()
total_count = sample_2.count()
normalized_counts = value_counts.withColumn('normalized_count', col('count') / total_count)

# Display the result
normalized_counts.show()

+-------+-----+----------------+
| estado|count|normalized_count|
+-------+-----+----------------+
|success|  150|             1.0|
+-------+-----+----------------+



El 100% de la muestra aleatoria del segundo dataframe (df_vivienda_usada_arriendo) se pudo georeferenciar.

In [0]:
sample_3 = df_comercio_base_proyecto_sp.sample(withReplacement=False, fraction=0.1, seed=1).limit(150)

In [0]:
sample_3 = sample_3.withColumn('geocode_result', struct(georeferenciar_udf(col(comercio_dir)).alias('geocoding')))

In [0]:
sample_3 = sample_3.withColumn('estado', col('geocode_result.geocoding.estado'))
sample_3 = sample_3.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
sample_3 = sample_3.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
sample_3 = sample_3.withColumn('lat', col('geocode_result.geocoding.yinput'))
sample_3 = sample_3.withColumn('lng', col('geocode_result.geocoding.xinput'))

In [0]:
# Calculate value counts and normalize
value_counts = sample_3.groupBy('estado').count()
total_count = sample_3.count()
normalized_counts = value_counts.withColumn('normalized_count', col('count') / total_count)

# Display the result
normalized_counts.show()

+--------------+-----+------------------+
|        estado|count|  normalized_count|
+--------------+-----+------------------+
|       success|   72|0.6666666666666666|
|        failed|   12|0.1111111111111111|
|outside_bogota|   24|0.2222222222222222|
+--------------+-----+------------------+



In [0]:
temp_data = sample_3.select(comercio_dir, 'estado').toPandas()
temp_data = pd.DataFrame(temp_data[temp_data['estado']=='failed'][comercio_dir])
temp_data

,Dirección
4,Cr. 101 Tr. Suba
8,Av. Cl. 80 # 66A-48
31,Cr. 85D # 51-65
34,Tr. 30 # 124-91
37,Av. 68 # 98-34
40,Av. Parque con cra 59
41,Av. Ciudad Cali con Esperanza
48,Av. Santa Fé Cl. 50 Sur
50,Av. Santa fé Cl. 53 F Sur Bosa
64,Av. Suba con Av. C. de Cali


El 11% de la muestra aleatoria del tercer dataframe (df_comercio_base_proyecto)  no pudo ser georeferenciada por las siguientes razones:
* Nombres de vías que no son aceptados por el API
* Detalles de texto en la dirección
* Referencia a lugares especificos (no direcciones)

Hay un 22% adicional que hace referencia a direcciones fuera de Bogotá (en otras ciudades o en vías nacionales)

In [0]:
sample_4 = df_comercio_base_inmuebles_sp.sample(withReplacement=False, fraction=0.1, seed=1).limit(150)

In [0]:
sample_4 = sample_4.withColumn('geocode_result', struct(georeferenciar_udf(col(comercio_dir)).alias('geocoding')))

In [0]:
sample_4 = sample_4.withColumn('estado', col('geocode_result.geocoding.estado'))
sample_4 = sample_4.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
sample_4 = sample_4.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
sample_4 = sample_4.withColumn('lat', col('geocode_result.geocoding.yinput'))
sample_4 = sample_4.withColumn('lng', col('geocode_result.geocoding.xinput'))

In [0]:
# Calculate value counts and normalize
value_counts = sample_4.groupBy('estado').count()
total_count = sample_4.count()
normalized_counts = value_counts.withColumn('normalized_count', col('count') / total_count)

# Display the result
normalized_counts.show()

+--------------+-----+-------------------+
|        estado|count|   normalized_count|
+--------------+-----+-------------------+
|       success|  104| 0.6933333333333334|
|outside_bogota|   38|0.25333333333333335|
|        failed|    8|0.05333333333333334|
+--------------+-----+-------------------+



In [0]:
temp_data = sample_4.select(comercio_dir, 'estado').toPandas()
temp_data = pd.DataFrame(temp_data[temp_data['estado']=='failed'][comercio_dir])
temp_data

,Dirección
24,Av. Cl. 80 # 66A-48
25,Av. Cl. 80 # 66A-48
26,Av. Cl. 80 # 66A-48
62,Dg. 76 # 23H-96F
66,A 900 mts de Aut. Medellín
67,A 900 mts de Aut. Medellín
68,A 900 mts de Aut. Medellín
69,A 900 mts de Aut. Medellín


El 5% de la muestra aleatoria del cuarto dataframe (df_comercio_base_inmuebles) no pudo ser georeferenciada por las siguientes razones:

* Nombres de vías que no son aceptados por el API
* Detalles de texto en la dirección
* Referencia a lugares especificos (no direcciones)

Hay un 38% adicional que hace referencia a direcciones fuera de Bogotá (en otras ciudades o en vías nacionales)

In [0]:
sample_5 = df_comercio_base_historia_sp.sample(withReplacement=False, fraction=0.1, seed=1).limit(150)

In [0]:
sample_5 = sample_5.withColumn('geocode_result', struct(georeferenciar_udf(col(comercio_dir)).alias('geocoding')))

In [0]:
sample_5 = sample_5.withColumn('estado', col('geocode_result.geocoding.estado'))
sample_5 = sample_5.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
sample_5 = sample_5.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
sample_5 = sample_5.withColumn('lat', col('geocode_result.geocoding.yinput'))
sample_5 = sample_5.withColumn('lng', col('geocode_result.geocoding.xinput'))

In [0]:
# Calculate value counts and normalize
value_counts = sample_5.groupBy('estado').count()
total_count = sample_5.count()
normalized_counts = value_counts.withColumn('normalized_count', col('count') / total_count)

# Display the result
normalized_counts.show()

+--------------+-----+------------------+
|        estado|count|  normalized_count|
+--------------+-----+------------------+
|       success|   64|0.5925925925925926|
|        failed|   17|0.1574074074074074|
|outside_bogota|   27|              0.25|
+--------------+-----+------------------+



In [0]:
temp_data = sample_5.select(comercio_dir, 'estado').toPandas()
temp_data = pd.DataFrame(temp_data[temp_data['estado']=='failed'][comercio_dir])
temp_data

,Dirección
1,Cr. 7 # 39-21
3,Av. Pradilla 2 Este-71
16,A 900 mts de Aut. Medellín
27,A 900 mts de Aut. Medellín
32,Cr. 85D # 51-65
41,Variante Chia-Cota frente a Parmalat
42,Av. Esperanza # 62-49
43,Cra 63 # 57G-46 Sur
48,Entrada Jaime Duque Entrada Ind.Crown
53,Av. 26 Al lado Puente áereo


El 15% de la muestra aleatoria del quinto dataframe (df_comercio_base_historia) no pudo ser georeferenciada por las siguientes razones:

* Nombres de vías que no son aceptados por el API
* Detalles de texto en la dirección
* Referencia a lugares especificos (no direcciones)

Hay un 27% adicional que hace referencia a direcciones fuera de Bogotá (en otras ciudades o en vías nacionales)

In [0]:
sample_6 = df_def_base_proyecto_sp.sample(withReplacement=False, fraction=0.1, seed=1).limit(150)

In [0]:
sample_6 = sample_6.withColumn('geocode_result', struct(georeferenciar_udf(col(def_dir)).alias('geocoding')))

In [0]:
sample_6 = sample_6.withColumn('estado', col('geocode_result.geocoding.estado'))
sample_6 = sample_6.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
sample_6 = sample_6.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
sample_6 = sample_6.withColumn('lat', col('geocode_result.geocoding.yinput'))
sample_6 = sample_6.withColumn('lng', col('geocode_result.geocoding.xinput'))

sample_6 = sample_6.withColumn("real_geom",create_point_udf(col("Longitud").cast(DoubleType()), col("Latitud (Coordenada Real)").cast(DoubleType()))
)

sample_6 = sample_6.withColumn("geocoded_geom",create_point_udf(col("lng").cast(DoubleType()), col("lat").cast(DoubleType()))
)

sample_6 = sample_6.withColumn("distance",distance_udf(col("real_geom"), col("geocoded_geom")))

In [0]:
# Calculate value counts and normalize
value_counts = sample_6.groupBy('estado').count()
total_count = sample_6.count()
normalized_counts = value_counts.withColumn('normalized_count', col('count') / total_count)

# Display the result
normalized_counts.show()

+--------------+-----+----------------+
|        estado|count|normalized_count|
+--------------+-----+----------------+
|       success|  138|            0.92|
|outside_bogota|    3|            0.02|
|        failed|    9|            0.06|
+--------------+-----+----------------+



In [0]:
temp_data = sample_6.select(def_dir, 'estado').toPandas()
temp_data = pd.DataFrame(temp_data[temp_data['estado']=='failed'][def_dir])
temp_data

,Dirección
10,Av. 127 Con Cr. 53
12,Cl. 134 # 8-32
43,Cl. 71 # 3-09
58,Cl. 75 # 6-62
72,Cl. 105 # 19 A 86
75,Av. 19 # 127B bis- 53
120,Cl. 128 # 7E- 40
146,Cl. 151 # 26-50
149,Vda. Canelón Stor. Rio Frio-Cajicá


In [0]:
sorted_distances = sample_6.filter(sample_6.estado == 'success').orderBy(col("distance").desc())
sorted_distances.select("distance").toPandas().plot.bar(figsize=(30,5))

El 6% de la muestra aleatoria del sexto dataframe (df_def_base_proyecto) no pudo ser georeferenciada por las siguientes razones:

* Nombres de vías que no son aceptados por el API
* Detalles de texto en la dirección
* Referencia a lugares especificos (no direcciones)

Hay un 2% adicional que hace referencia a direcciones fuera de Bogotá (en otras ciudades o en vías nacionales)

Adicionalmente, en este dataframe se contaba con coordenadas reales de referencia. En la revisión de los datos, el geocodificador muestra una alta presición. El 19% de los casos se ubicaron a mas de 100 metros de la coordenada original.

## Procesamiento

In [0]:
df_vivienda_usada_venta_sp = df_vivienda_usada_venta_sp.withColumn('geocode_result', struct(georeferenciar_udf(col(vivienda_usada_dir)).alias('geocoding')))

In [0]:
df_vivienda_usada_venta_sp = df_vivienda_usada_venta_sp.withColumn('estado', col('geocode_result.geocoding.estado'))
df_vivienda_usada_venta_sp = df_vivienda_usada_venta_sp.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
df_vivienda_usada_venta_sp = df_vivienda_usada_venta_sp.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
df_vivienda_usada_venta_sp = df_vivienda_usada_venta_sp.withColumn('lat', col('geocode_result.geocoding.yinput'))
df_vivienda_usada_venta_sp = df_vivienda_usada_venta_sp.withColumn('lng', col('geocode_result.geocoding.xinput'))


In [0]:
# Calculate value counts and normalize
value_counts = df_vivienda_usada_venta_sp.groupBy('estado').count()
total_count = df_vivienda_usada_venta_sp.count()
normalized_counts = value_counts.withColumn('normalized_count', col('count') / total_count)

# Display the result
normalized_counts.show()

+--------+-----+--------------------+
|  estado|count|    normalized_count|
+--------+-----+--------------------+
| success|43277|  0.9894147233653406|
|  failed|  424|0.009693644261545497|
|error_ws|   39|8.916323731138546E-4|
+--------+-----+--------------------+



In [0]:
df_vivienda_usada_venta_sp = df_vivienda_usada_venta_sp.drop("geocode_result")

Out[543]: DataFrame[Consecutivo: string, Nombre Proyecto: string, Recorrido: string, Dirección Nueva: string, Subzona: string, Barrio: string, Estrato: string, Area: string, Valor Venta: string, Antigüedad: string, Subtipo: string, Es interior o exterior (Interior/Exterior): string, Si es apto en que piso está: string, Alcobas: string, # Garajes: string, Area Terraza: string, Costo Admon: string, Fecha Inicio Negocio (dia/mes/año): string, Tipo Negocio Cerrado (Vendido/Arrendado/Desis): string, Fecha Actualizacion (dia/mes/año): string, Tipo Negocio (V A  V/A): string, Dias definicion: string, Telefono 1: string, Telefono 2: string, Telefono 3: string, Nombre Inmobiliaria 1: string, $m2: string, $m2/Nuevo: string, $m2/Año Deseado: string, Costo Admon. M2: string, Mes/año cierre: string, Año Inicial Negocio: string, estado: string, tipo_direccion: string, diraprox: string, lat: double, lng: double]

In [0]:
permanent_table_name = "`df_vivienda_usada_venta_sp`"
df_vivienda_usada_venta_sp.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
df_vivienda_usada_arriendo_sp = df_vivienda_usada_arriendo_sp.withColumn('geocode_result', struct(georeferenciar_udf(col(vivienda_usada_dir)).alias('geocoding')))

In [0]:
df_vivienda_usada_arriendo_sp = df_vivienda_usada_arriendo_sp.withColumn('estado', col('geocode_result.geocoding.estado'))
df_vivienda_usada_arriendo_sp = df_vivienda_usada_arriendo_sp.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
df_vivienda_usada_arriendo_sp = df_vivienda_usada_arriendo_sp.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
df_vivienda_usada_arriendo_sp = df_vivienda_usada_arriendo_sp.withColumn('lat', col('geocode_result.geocoding.yinput'))
df_vivienda_usada_arriendo_sp = df_vivienda_usada_arriendo_sp.withColumn('lng', col('geocode_result.geocoding.xinput'))


In [0]:
# Calculate value counts and normalize
value_counts = df_vivienda_usada_arriendo_sp.groupBy('estado').count()
total_count = df_vivienda_usada_arriendo_sp.count()
normalized_counts = value_counts.withColumn('normalized_count', col('count') / total_count)

# Display the result
normalized_counts.show()

+--------+-----+--------------------+
|  estado|count|    normalized_count|
+--------+-----+--------------------+
| success|37918|  0.9900002610897888|
|  failed|  349|0.009112033628364795|
|error_ws|   34| 8.87705281846427E-4|
+--------+-----+--------------------+



In [0]:
df_vivienda_usada_arriendo_sp = df_vivienda_usada_arriendo_sp.drop("geocode_result")

In [0]:
permanent_table_name = "`df_vivienda_usada_arriendo_sp`"
df_vivienda_usada_arriendo_sp.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
df_comercio_base_proyecto_sp = df_comercio_base_proyecto_sp.withColumn('geocode_result', struct(georeferenciar_udf(col(comercio_dir)).alias('geocoding')))

In [0]:
df_comercio_base_proyecto_sp = df_comercio_base_proyecto_sp.withColumn('estado', col('geocode_result.geocoding.estado'))
df_comercio_base_proyecto_sp = df_comercio_base_proyecto_sp.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
df_comercio_base_proyecto_sp = df_comercio_base_proyecto_sp.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
df_comercio_base_proyecto_sp = df_comercio_base_proyecto_sp.withColumn('lat', col('geocode_result.geocoding.yinput'))
df_comercio_base_proyecto_sp = df_comercio_base_proyecto_sp.withColumn('lng', col('geocode_result.geocoding.xinput'))


In [0]:
# Calculate value counts and normalize
value_counts = df_comercio_base_proyecto_sp.groupBy('estado').count()
total_count = df_comercio_base_proyecto_sp.count()
normalized_counts = value_counts.withColumn('normalized_count', col('count') / total_count)

# Display the result
normalized_counts.show()

+--------------+-----+--------------------+
|        estado|count|    normalized_count|
+--------------+-----+--------------------+
|       success|  701|  0.6545284780578898|
|        failed|  134| 0.12511671335200747|
|      error_ws|    1|9.337068160597573E-4|
|outside_bogota|  235| 0.21942110177404295|
+--------------+-----+--------------------+



In [0]:
df_comercio_base_proyecto_sp = df_comercio_base_proyecto_sp.drop("geocode_result")

In [0]:
permanent_table_name = "`df_comercio_base_proyecto_sp_2`"
df_comercio_base_proyecto_sp.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
df_comercio_base_inmuebles_sp = df_comercio_base_inmuebles_sp.withColumn('geocode_result', struct(georeferenciar_udf(col(comercio_dir)).alias('geocoding')))

In [0]:
df_comercio_base_inmuebles_sp = df_comercio_base_inmuebles_sp.withColumn('estado', col('geocode_result.geocoding.estado'))
df_comercio_base_inmuebles_sp = df_comercio_base_inmuebles_sp.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
df_comercio_base_inmuebles_sp = df_comercio_base_inmuebles_sp.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
df_comercio_base_inmuebles_sp = df_comercio_base_inmuebles_sp.withColumn('lat', col('geocode_result.geocoding.yinput'))
df_comercio_base_inmuebles_sp = df_comercio_base_inmuebles_sp.withColumn('lng', col('geocode_result.geocoding.xinput'))

In [0]:
# Calculate value counts and normalize
value_counts = df_comercio_base_inmuebles_sp.groupBy('estado').count()
total_count = df_comercio_base_inmuebles_sp.count()
normalized_counts = value_counts.withColumn('normalized_count', col('count') / total_count)

# Display the result
normalized_counts.show()

+--------------+-----+--------------------+
|        estado|count|    normalized_count|
+--------------+-----+--------------------+
|       success| 5051|  0.6599163835902796|
|        failed|  877| 0.11458061144499608|
|      error_ws|    2|2.613012803762738E-4|
|outside_bogota| 1724| 0.22524170368434807|
+--------------+-----+--------------------+



In [0]:
df_comercio_base_inmuebles_sp = df_comercio_base_inmuebles_sp.drop("geocode_result")

In [0]:
permanent_table_name = "`df_comercio_base_inmuebles_sp`"
df_comercio_base_inmuebles_sp.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
df_comercio_base_historia_sp = df_comercio_base_historia_sp.withColumn('geocode_result', struct(georeferenciar_udf(col(comercio_dir)).alias('geocoding')))

In [0]:
df_comercio_base_historia_sp = df_comercio_base_historia_sp.withColumn('estado', col('geocode_result.geocoding.estado'))
df_comercio_base_historia_sp = df_comercio_base_historia_sp.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
df_comercio_base_historia_sp = df_comercio_base_historia_sp.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
df_comercio_base_historia_sp = df_comercio_base_historia_sp.withColumn('lat', col('geocode_result.geocoding.yinput'))
df_comercio_base_historia_sp = df_comercio_base_historia_sp.withColumn('lng', col('geocode_result.geocoding.xinput'))

In [0]:
# Calculate value counts and normalize
value_counts = df_comercio_base_historia_sp.groupBy('estado').count()
total_count = df_comercio_base_historia_sp.count()
normalized_counts = value_counts.withColumn('normalized_count', col('count') / total_count)

# Display the result
normalized_counts.show()

+--------------+-----+--------------------+
|        estado|count|    normalized_count|
+--------------+-----+--------------------+
|       success|  701|  0.6545284780578898|
|        failed|  134| 0.12511671335200747|
|      error_ws|    1|9.337068160597573E-4|
|outside_bogota|  235| 0.21942110177404295|
+--------------+-----+--------------------+



In [0]:
df_comercio_base_historia_sp = df_comercio_base_historia_sp.drop("geocode_result")

In [0]:
permanent_table_name = "`df_comercio_base_historia_sp`"
df_comercio_base_historia_sp.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
df_def_base_proyecto_sp = df_def_base_proyecto_sp.withColumn('geocode_result', struct(georeferenciar_udf(col(def_dir)).alias('geocoding')))

In [0]:
df_def_base_proyecto_sp = df_def_base_proyecto_sp.withColumn('estado', col('geocode_result.geocoding.estado'))
df_def_base_proyecto_sp = df_def_base_proyecto_sp.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
df_def_base_proyecto_sp = df_def_base_proyecto_sp.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
df_def_base_proyecto_sp = df_def_base_proyecto_sp.withColumn('lat', col('geocode_result.geocoding.yinput'))
df_def_base_proyecto_sp = df_def_base_proyecto_sp.withColumn('lng', col('geocode_result.geocoding.xinput'))

#df_def_base_proyecto_sp = df_def_base_proyecto_sp.withColumn("real_geom",create_point_udf(col("Longitud").cast(DoubleType()), col("Latitud (Coordenada Real)").cast(DoubleType())))

#df_def_base_proyecto_sp = df_def_base_proyecto_sp.withColumn("geocoded_geom",create_point_udf(col("lng").cast(DoubleType()), col("lat").cast(DoubleType())))

#df_def_base_proyecto_sp = df_def_base_proyecto_sp.withColumn("distance",distance_udf(col("real_geom"), col("geocoded_geom")))

In [0]:
# Calculate value counts and normalize
value_counts = df_def_base_proyecto_sp.groupBy('estado').count()
total_count = df_def_base_proyecto_sp.count()
normalized_counts = value_counts.withColumn('normalized_count', col('count') / total_count)

# Display the result
normalized_counts.show()

+--------------+-----+-------------------+
|        estado|count|   normalized_count|
+--------------+-----+-------------------+
|       success| 3717| 0.6997364457831325|
|        failed|  757|0.14250753012048192|
|outside_bogota|  838|0.15775602409638553|
+--------------+-----+-------------------+



In [0]:
df_def_base_proyecto_sp = df_def_base_proyecto_sp.drop("geocode_result")

In [0]:
permanent_table_name = "`df_def_base_proyecto_sp`"
df_def_base_proyecto_sp.write.format("parquet").saveAsTable(permanent_table_name)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3330639135281278>:2
      1 permanent_table_name = "`df_def_base_proyecto_sp`"
----> 2 df_def_base_proyecto_sp.write.format("parquet").saveAsTable(permanent_table_name)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1520, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1518 if format is not None:
   1519     self.format(format)
-> 1520 self._jwrite.saveAsTable(name)

File /databricks/spark/python/lib/py4j-0.10.9

In [0]:
filename = '/dbfs/FileStore/tables/galeria/df_vivienda_usada_venta_sp.csv'
df_vivienda_usada_venta_sp.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save(filename)

In [0]:
filename = '/dbfs/FileStore/tables/galeria/df_vivienda_usada_arriendo_sp.csv'
df_vivienda_usada_arriendo_sp.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save(filename)

In [0]:
filename = '/dbfs/FileStore/tables/galeria/df_comercio_base_proyecto_sp.csv'
df_comercio_base_proyecto_sp.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save(filename)

In [0]:
filename = '/dbfs/FileStore/tables/galeria/df_comercio_base_inmuebles_sp.csv'
df_comercio_base_inmuebles_sp.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save(filename)

In [0]:
filename = '/dbfs/FileStore/tables/galeria/df_comercio_base_historia_sp.csv'
df_comercio_base_historia_sp.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save(filename)

In [0]:
filename = '/dbfs/FileStore/tables/galeria/df_def_base_proyecto_sp.csv'
df_def_base_proyecto_sp.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save(filename)

In [0]:
%ls '/FileStore/tables/'

ls: cannot access '/FileStore/tables/': No such file or directory
